In [110]:
# neural network of many complex valued neurons in 3 layers
# applied to the MNIST dataset
import numpy

In [111]:
class neuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, cats, periods):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes +1 # increament for bias node
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.uniform(-1.0, 1.0, (self.hnodes, self.inodes))
        self.wih = numpy.array(self.wih, ndmin=2, dtype='complex128')
        self.wih += 1j * numpy.random.uniform(-1.0, 1.0, (self.hnodes, self.inodes))
        
        self.who = numpy.random.uniform(-1.0, 1.0, (self.onodes, self.hnodes))
        self.who = numpy.array(self.who, ndmin=2, dtype='complex128')
        self.who += 1j * numpy.random.random((self.onodes, self.hnodes))
        
        # number of output class categories
        self.categories = cats
        
        # todo periodicity
        self.periodicity = periods
        pass
    
    def z_to_class(self, z):
        # first work out the angle, but shift angle from [-pi/2, +pi.2] to [0,2pi]
        angle = numpy.mod(numpy.angle(z) + 2*numpy.pi, 2*numpy.pi)
        # from angle to category
        p = int(numpy.floor (self.categories * angle / (2*numpy.pi)))
        return p

    def class_to_angle(self, p):
        # class to angle, using bisector
        angle = ((p + 0.5) / self.categories) * 2 * numpy.pi
        return angle
    
    def status(self):
        print ("self.wih = ", self.wih)
        print ("self.who = ", self.who)
        pass

    def query(self, inputs_list):
        # add bias input
        inputs_list.append(1.0)
        
        # convert input to complex
        inputs = numpy.array(inputs_list, ndmin=2, dtype='complex128').T
        #print("inputs = \n", inputs)
        
        # signal into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #print("hidden_inputs = ", hidden_inputs)
        #signal out of hidden layer
        hidden_outputs = numpy.exp(1j * numpy.angle(hidden_inputs))
        #print("hidden_outputs = ", hidden_outputs)
        
        # signal into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #print("final_input = ", final_inputs)
        #signal out of output layer
        final_outputs = numpy.exp(1j * numpy.angle(final_inputs))
        #print("final_outputs = ", final_outputs)
        
        # map to output classes
        output_classes = self.z_to_class(final_outputs)
        #print("output_classes = ", output_classes)
        return output_classes
    
    def train(self, inputs_list, target_class_list):
        # add bias input
        inputs_list.append(1.0)
        
        # convert input to complex
        inputs = numpy.array(inputs_list, ndmin=2, dtype='complex128').T
        #print("inputs = \n", inputs)
        
        # map target classes to unit circle
        targets = numpy.exp(1j * self.class_to_angle(numpy.array(target_class_list, ndmin=2).T))
        #print("targets = \n", targets)

        # signal into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #print("hidden_inputs = ", hidden_inputs)
        #signal out of hidden layer
        hidden_outputs = numpy.exp(1j * numpy.angle(hidden_inputs))
        #print("hidden_outputs = ", hidden_outputs)
        
        # signal into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #print("final_inputs = ", final_inputs)
        #signal out of output layer
        final_outputs = numpy.exp(1j * numpy.angle(final_inputs))
        #print("final_outputs = ", final_outputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        #print("output_errors = ", output_errors)
        # hidden layer error is the output_errors split simply (not by weights)
        hidden_errors = numpy.dot(numpy.array(numpy.ones((self.hnodes)), ndmin=2).T / self.hnodes, output_errors)
        #print("hidden_errors = ", hidden_errors)
        
        # dw = e * x.T / (x.x.T)
        dwho = numpy.dot(output_errors, numpy.conj(hidden_outputs.T)) / (self.hnodes)
        #print("dwho = ", dwho)
        self.who += dwho
        
        dwih = numpy.dot(hidden_errors, numpy.conj(inputs.T)) / (self.inodes)
        #print("dwih = ", dwih)
        self.wih += dwih        
    pass

In [112]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 100
output_nodes = 1

# categories, periodicity
categories = 10
periodicity = 1

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, categories, periodicity)

In [113]:
# load the mnist training data CSV file into a list
training_data_file = open("mnist_dataset/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [114]:
# train neural network - OR

epochs = 1

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = numpy.exp(1j * (numpy.asfarray(all_values[1:]) / 255.0) * numpy.pi)
        inputs_list = inputs.tolist()
        # create the target class list
        targets = [int(all_values[0])]
        n.train(inputs_list, targets)
        pass
    pass

In [115]:
# load the mnist test data CSV file into a list
test_data_file = open("mnist_dataset/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [116]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = numpy.exp(1j * (numpy.asfarray(all_values[1:]) / 255.0) * numpy.pi)
    inputs_list = inputs.tolist()
    # query the network
    output_label = n.query(inputs_list)
    # append correct or incorrect to list
    if (output_label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

In [117]:
# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.1719
